In [1]:
import tensorflow as tf
import numpy as np
import gym
import os
import shutil
from time import time

In [2]:
# значения констант взяты отсюда https://github.com/openai/gym/blob/master/gym/envs/box2d/bipedal_walker.py
SCALE  = 30.0 
TERRAIN_STEP   = 14/SCALE
TERRAIN_LENGTH = 200     # in steps
TERRAIN_GRASS    = 10    # low long are grass spots, in steps

## Для воспроизводимых результатов
потому что они ни разу не воспроизводятся, а я не понимаю, какой еще сид надо фиксировать, кроме этих двух


In [3]:
np.random.seed(16)
tf.set_random_seed(16)

## Параметры:

In [4]:
max_episodes = 1100
lr_actor = 0.0005
lr_critic = 0.0005
gamma = 0.999 
memory_capacity = 200000
batch_size = 32
render_threshold = 150
url = './data'
render = False
env_name = 'BipedalWalker-v2'

## Среда и ее параметры

In [5]:
env = gym.make(env_name)
env.seed(1)

state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0] 
action_bound = env.action_space.high

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


## Memory

In [6]:
class Memory(object):
    
    @property
    def root_priority(self):
        return self.tree[0]

    eps = 0.001
    alpha = 0.6 
    beta = 0.4
    beta_inc = 1e-5
    abs_err = 1
    data_pointer=0

    def __init__(self, capacity):
        self.capacity = capacity
        self.tree = np.zeros(2 * capacity - 1)+1e-5
        self.data = np.zeros(capacity, dtype=object)

    def remember(self, error, data):
        p = self._get_priority(error)
        idx = self.data_pointer + self.capacity - 1
        self.data[self.data_pointer] = data
        change = p - self.tree[idx]
        self.tree[idx] = p
        self._propagate_change(idx, change)
        self.data_pointer += 1
        if self.data_pointer >= self.capacity:
            self.data_pointer = 0

    def sample(self, n):
        batch_idx, batch_memory, weights = [], [], []
        segment = self.root_priority / n
        self.beta = np.min([1, self.beta + self.beta_inc])

        min_prob = np.min(self.tree[-self.capacity:]) / self.root_priority
        maxiwi = np.power(self.capacity * min_prob, -self.beta)
        for i in range(n):
            a = segment * i
            b = segment * (i + 1)
            lower_bound = np.random.uniform(a, b)
            while True:
                idx = self._recover(lower_bound)
                data_idx = idx - self.capacity + 1
                data = self.data[data_idx]
                p = self.tree[idx]
                if type(data) is int:
                    i -= 1
                    lower_bound = np.random.uniform(segment * i, segment * (i+1))
                else:
                    break
            prob = p / self.root_priority
            weights.append(self.capacity * prob)
            batch_idx.append(idx)
            batch_memory.append(data)

        weights = np.vstack(weights)
        weights = np.power(weights, -self.beta) / maxiwi
        return batch_idx, np.vstack(batch_memory), weights

    def update(self, idx, error):
        p = self._get_priority(error)
        change = p - self.tree[idx]
        self.tree[idx] = p
        self._propagate_change(idx, change)

    def _get_priority(self, error):
        error += self.eps
        c_error = np.clip(error, 0, self.abs_err)
        return np.power(c_error, self.alpha)

    def _propagate_change(self, tree_idx, change):
        parent_idx = (tree_idx - 1) // 2
        self.tree[parent_idx] += change
        if parent_idx != 0:
            self._propagate_change(parent_idx, change)

    
    def _recover(self, lower_bound, parent_i=0):

        lchild_i = 2 * parent_i + 1
        rchild_i = lchild_i + 1

        if lchild_i >= len(self.tree):
            return parent_i

        if self.tree[lchild_i] == self.tree[rchild_i]:
            return self._recover(lower_bound, np.random.choice([lchild_i, rchild_i]))
        if lower_bound <= self.tree[lchild_i]:
            return self._recover(lower_bound, lchild_i)
        else:
            return self._recover(lower_bound - self.tree[lchild_i], rchild_i)




## Placeholder'ы и другие сущности tensorflow

In [7]:
global_step = tf.Variable(0, trainable=False)
increase_gs = global_step.assign(tf.add(global_step, 1))
lr_actor = tf.train.exponential_decay(lr_actor, global_step, 10000, .97, staircase=True)
lr_critic = tf.train.exponential_decay(lr_critic, global_step, 10000, .97, staircase=True)

STATE = tf.placeholder(tf.float32, shape=[None, state_dim], name='state')
REWARD = tf.placeholder(tf.float32, [None, 1], name='reward')
STATE_ = tf.placeholder(tf.float32, shape=[None, state_dim], name='state_')

## Actor

In [8]:
class Actor(object):
    def __init__(self, sess, action_dim, action_bound, learning_rate, replace_iter):
        self.sess = sess
        self.action_dim = action_dim
        self.action_bound = action_bound
        self.learning_rate = learning_rate
        self.replace_iter = replace_iter
        self.replace_counter = 0

        with tf.variable_scope('Actor'):
            self.action = self._build_net(STATE, scope='eval_net', trainable=True)
            self.action_ = self._build_net(STATE_, scope='target_net', trainable=False)

        self.e_params = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='Actor/eval_net')
        self.t_params = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='Actor/target_net')

    def _build_net(self, state, scope, trainable):
        with tf.variable_scope(scope):
            init_weights = tf.random_normal_initializer(0., 0.01)
            init_bias = tf.constant_initializer(0.01)
            net = tf.layers.dense(state, 500, activation=tf.nn.relu,
                                  kernel_initializer=init_weights,
                                  bias_initializer=init_bias, name='l1', trainable=trainable)
            net = tf.layers.dense(net, 200, activation=tf.nn.relu,
                                  kernel_initializer=init_weights,
                                  bias_initializer=init_bias,
                                  name='l2',
                                  trainable=trainable)

            actions = tf.layers.dense(net, self.action_dim,
                                      activation=tf.nn.tanh,
                                      kernel_initializer=init_weights,
                                      bias_initializer=init_bias,
                                      name='actions',
                                      trainable=trainable)
            scaled_actions = tf.multiply(actions, self.action_bound, name='scaled_actions')
        return scaled_actions

    def learn(self, state):
        self.sess.run(self.train_op, feed_dict={STATE: state})
        if self.replace_counter % self.replace_iter == 0:
            self.sess.run([tf.assign(t, e) for t, e in zip(self.t_params, self.e_params)])
        self.replace_counter += 1

    def choose_action(self, state):
        state = state[np.newaxis, :]
        return self.sess.run(self.action, feed_dict={STATE: state})[0]

## Critic

In [9]:
class Critic(object):
    def __init__(self, sess, state_dim, action_dim, learning_rate, gamma, replace_iter, action, action_):
        self.sess = sess
        self.s_dim = state_dim
        self.action_dim = action_dim
        self.gamma = gamma
        self.replace_iter = replace_iter
        self.replace_counter = 0
        self.learning_rate = learning_rate
        self.weights = tf.placeholder(tf.float32, [None, 1], name='IS_weights')

        with tf.variable_scope('Critic'):
            self.action = action
            self.q = self._build_net(STATE, self.action, 'eval_net', trainable=True)
            self.q_ = self._build_net(STATE_, action_, 'target_net', trainable=False)
            self.t_params = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='Critic/target_net')
            self.e_params = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='Critic/eval_net')
            self.target_q = REWARD + self.gamma * self.q_
            self.abs_td = tf.abs(self.target_q - self.q)
            self.loss = tf.reduce_mean(self.weights * tf.squared_difference(self.target_q, self.q))
            self.train_op = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss, global_step=global_step)
            self.actor_grads = tf.gradients(self.q, action)[0]

    def _build_net(self, state, action, scope, trainable):
        with tf.variable_scope(scope):
            init_weights = tf.random_normal_initializer(0., 0.01)
            init_bias = tf.constant_initializer(0.01)

            with tf.variable_scope('l1'):
                n_l1 = 700
                w1_state = tf.get_variable('w1_state', [self.s_dim, n_l1],
                                           initializer=init_weights, trainable=trainable)
                w1_action = tf.get_variable('w1_action', [self.action_dim, n_l1],
                                            initializer=init_weights, trainable=trainable)
                b1 = tf.get_variable('b1', [1, n_l1], initializer=init_bias, trainable=trainable)
                net = tf.nn.relu(tf.matmul(state, w1_state) + tf.matmul(action, w1_action) + b1)
            with tf.variable_scope('l2'):
                net = tf.layers.dense(net, 20, activation=tf.nn.relu, kernel_initializer=init_weights,
                                      bias_initializer=init_bias, name='l2', trainable=trainable)
            with tf.variable_scope('q'):
                q = tf.layers.dense(net, 1, kernel_initializer=init_weights,
                                    bias_initializer=init_bias, trainable=trainable)
        return q

    def learn(self, state, action, reward, state_, weights):
        _, abs_td = self.sess.run([self.train_op, self.abs_td], feed_dict={STATE: state,
                                                                           self.action: action,
                                                                           REWARD: reward,
                                                                           STATE_: state_,
                                                                           self.weights: weights})
        if self.replace_counter % self.replace_iter == 0:
            self.sess.run([tf.assign(t, e) for t, e in zip(self.t_params, self.e_params)])
        self.replace_counter += 1
        return abs_td


## Создаем actor'a и critic'a

In [10]:
sess = tf.Session()

actor = Actor(sess, action_dim, action_bound, lr_actor, 1700)
critic = Critic(sess, state_dim, action_dim, lr_critic, gamma, 1500, actor.action, actor.action_)

with tf.variable_scope('policy_grads'):
    policy_grads_and_vars = tf.gradients(actor.action, xs=actor.e_params, grad_ys=critic.actor_grads)
with tf.variable_scope('A_train'):
    opt = tf.train.RMSPropOptimizer(-actor.learning_rate)
    actor.train_op = opt.apply_gradients(zip(policy_grads_and_vars, actor.e_params), global_step=global_step)

In [11]:
M = Memory(memory_capacity)
saver = tf.train.Saver(max_to_keep=100)

## Note
! Сеть обучается по-разному каждый раз, мне не удалось получить воспроизводимые результаты, даже фиксируя сиды numpy и tensorflow.

In [13]:
sess.run(tf.global_variables_initializer())

sigma = 3
sigma_min = 0.01
running_reward = 0

for i_episode in range(max_episodes):

    start = time()
    state = env.reset()
    step = 0
    actual_reward = 0

    while True:
        if render:
            env.render()
        action = actor.choose_action(state)
        action = np.clip(np.random.normal(action, sigma), -1, 1)
        state_, reward, done, _ = env.step(action)
        
        actual_reward += reward
        step += 1
        
        result = "----"
        if reward==-100:
            result = "FAILED"
        elif env.unwrapped.hull.position[0] > (TERRAIN_LENGTH-TERRAIN_GRASS)*TERRAIN_STEP:
            result = "SUCCEEDED"
        if reward == -100: 
            reward = -2

        experience = np.hstack((state, action, [reward], state_))
        max_p = np.max(M.tree[-M.capacity:])
        M.remember(max_p, experience)

        if global_step.eval(sess) > memory_capacity/20:
            sigma = max([sigma*0.9999, sigma_min])
            idxs, batch_memory, weights = M.sample(batch_size)
            batch_state = batch_memory[:, :state_dim]
            batch_action = batch_memory[:, state_dim: state_dim + action_dim]
            batch_reward = batch_memory[:, -state_dim - 1: -state_dim]
            batch_state_ = batch_memory[:, -state_dim:]

            abs_td = critic.learn(batch_state, batch_action, batch_reward, batch_state_, weights)
            actor.learn(batch_state)
            
            for i in range(len(idxs)):
                idx = idxs[i]
                M.update(idx, abs_td[i])

        if done:
            if actual_reward > render_threshold:
                render = True
            else: 
                render = False
            
           
            print('Episode:', i_episode,
                  '--- Time: %.3f' % (time() - start),
                  '--- Reward: %i' % int(actual_reward),
                  '--- Result: %s' % result)
            break

        state = state_
        sess.run(increase_gs)

Episode: 0 --- Time: 0.239 --- Reward: -135 --- Result: FAILED
Episode: 1 --- Time: 0.049 --- Reward: -105 --- Result: FAILED
Episode: 2 --- Time: 1.276 --- Reward: -157 --- Result: ----
Episode: 3 --- Time: 0.058 --- Reward: -104 --- Result: FAILED
Episode: 4 --- Time: 0.046 --- Reward: -114 --- Result: FAILED
Episode: 5 --- Time: 0.048 --- Reward: -116 --- Result: FAILED
Episode: 6 --- Time: 0.067 --- Reward: -108 --- Result: FAILED
Episode: 7 --- Time: 0.044 --- Reward: -105 --- Result: FAILED
Episode: 8 --- Time: 1.281 --- Reward: -155 --- Result: ----
Episode: 9 --- Time: 0.061 --- Reward: -112 --- Result: FAILED
Episode: 10 --- Time: 0.053 --- Reward: -114 --- Result: FAILED
Episode: 11 --- Time: 0.052 --- Reward: -115 --- Result: FAILED
Episode: 12 --- Time: 0.155 --- Reward: -137 --- Result: FAILED
Episode: 13 --- Time: 0.078 --- Reward: -102 --- Result: FAILED
Episode: 14 --- Time: 0.703 --- Reward: -208 --- Result: FAILED
Episode: 15 --- Time: 0.037 --- Reward: -110 --- Resul

Episode: 129 --- Time: 0.714 --- Reward: -103 --- Result: FAILED
Episode: 130 --- Time: 0.601 --- Reward: -101 --- Result: FAILED
Episode: 131 --- Time: 0.818 --- Reward: -127 --- Result: FAILED
Episode: 132 --- Time: 0.249 --- Reward: -102 --- Result: FAILED
Episode: 133 --- Time: 0.452 --- Reward: -100 --- Result: FAILED
Episode: 134 --- Time: 0.345 --- Reward: -97 --- Result: FAILED
Episode: 135 --- Time: 0.326 --- Reward: -103 --- Result: FAILED
Episode: 136 --- Time: 0.244 --- Reward: -102 --- Result: FAILED
Episode: 137 --- Time: 0.301 --- Reward: -102 --- Result: FAILED
Episode: 138 --- Time: 0.315 --- Reward: -105 --- Result: FAILED
Episode: 139 --- Time: 0.277 --- Reward: -105 --- Result: FAILED
Episode: 140 --- Time: 0.334 --- Reward: -104 --- Result: FAILED
Episode: 141 --- Time: 0.357 --- Reward: -99 --- Result: FAILED
Episode: 142 --- Time: 0.486 --- Reward: -98 --- Result: FAILED
Episode: 143 --- Time: 0.306 --- Reward: -99 --- Result: FAILED
Episode: 144 --- Time: 0.408 

Episode: 256 --- Time: 0.377 --- Reward: -98 --- Result: FAILED
Episode: 257 --- Time: 0.399 --- Reward: -99 --- Result: FAILED
Episode: 258 --- Time: 0.443 --- Reward: -102 --- Result: FAILED
Episode: 259 --- Time: 0.385 --- Reward: -99 --- Result: FAILED
Episode: 260 --- Time: 0.356 --- Reward: -99 --- Result: FAILED
Episode: 261 --- Time: 0.351 --- Reward: -100 --- Result: FAILED
Episode: 262 --- Time: 0.496 --- Reward: -95 --- Result: FAILED
Episode: 263 --- Time: 0.338 --- Reward: -101 --- Result: FAILED
Episode: 264 --- Time: 0.882 --- Reward: -106 --- Result: FAILED
Episode: 265 --- Time: 0.566 --- Reward: -97 --- Result: FAILED
Episode: 266 --- Time: 0.498 --- Reward: -101 --- Result: FAILED
Episode: 267 --- Time: 0.434 --- Reward: -99 --- Result: FAILED
Episode: 268 --- Time: 0.379 --- Reward: -99 --- Result: FAILED
Episode: 269 --- Time: 0.345 --- Reward: -98 --- Result: FAILED
Episode: 270 --- Time: 0.431 --- Reward: -99 --- Result: FAILED
Episode: 271 --- Time: 0.420 --- Re

Episode: 383 --- Time: 0.386 --- Reward: -99 --- Result: FAILED
Episode: 384 --- Time: 0.257 --- Reward: -103 --- Result: FAILED
Episode: 385 --- Time: 0.397 --- Reward: -105 --- Result: FAILED
Episode: 386 --- Time: 0.467 --- Reward: -101 --- Result: FAILED
Episode: 387 --- Time: 0.425 --- Reward: -101 --- Result: FAILED
Episode: 388 --- Time: 0.506 --- Reward: -101 --- Result: FAILED
Episode: 389 --- Time: 0.428 --- Reward: -101 --- Result: FAILED
Episode: 390 --- Time: 0.459 --- Reward: -102 --- Result: FAILED
Episode: 391 --- Time: 0.437 --- Reward: -104 --- Result: FAILED
Episode: 392 --- Time: 0.414 --- Reward: -99 --- Result: FAILED
Episode: 393 --- Time: 0.417 --- Reward: -100 --- Result: FAILED
Episode: 394 --- Time: 0.306 --- Reward: -103 --- Result: FAILED
Episode: 395 --- Time: 0.426 --- Reward: -100 --- Result: FAILED
Episode: 396 --- Time: 0.498 --- Reward: -98 --- Result: FAILED
Episode: 397 --- Time: 0.392 --- Reward: -103 --- Result: FAILED
Episode: 398 --- Time: 0.497

Episode: 510 --- Time: 0.342 --- Reward: -98 --- Result: FAILED
Episode: 511 --- Time: 0.425 --- Reward: -99 --- Result: FAILED
Episode: 512 --- Time: 0.415 --- Reward: -99 --- Result: FAILED
Episode: 513 --- Time: 0.369 --- Reward: -100 --- Result: FAILED
Episode: 514 --- Time: 0.360 --- Reward: -99 --- Result: FAILED
Episode: 515 --- Time: 0.369 --- Reward: -98 --- Result: FAILED
Episode: 516 --- Time: 0.415 --- Reward: -101 --- Result: FAILED
Episode: 517 --- Time: 0.350 --- Reward: -96 --- Result: FAILED
Episode: 518 --- Time: 0.419 --- Reward: -100 --- Result: FAILED
Episode: 519 --- Time: 0.448 --- Reward: -101 --- Result: FAILED
Episode: 520 --- Time: 0.371 --- Reward: -99 --- Result: FAILED
Episode: 521 --- Time: 0.407 --- Reward: -97 --- Result: FAILED
Episode: 522 --- Time: 0.360 --- Reward: -100 --- Result: FAILED
Episode: 523 --- Time: 0.340 --- Reward: -100 --- Result: FAILED
Episode: 524 --- Time: 0.394 --- Reward: -101 --- Result: FAILED
Episode: 525 --- Time: 0.365 --- 

Episode: 637 --- Time: 0.447 --- Reward: -102 --- Result: FAILED
Episode: 638 --- Time: 0.424 --- Reward: -102 --- Result: FAILED
Episode: 639 --- Time: 0.417 --- Reward: -98 --- Result: FAILED
Episode: 640 --- Time: 0.427 --- Reward: -100 --- Result: FAILED
Episode: 641 --- Time: 0.441 --- Reward: -102 --- Result: FAILED
Episode: 642 --- Time: 0.466 --- Reward: -97 --- Result: FAILED
Episode: 643 --- Time: 0.422 --- Reward: -97 --- Result: FAILED
Episode: 644 --- Time: 0.467 --- Reward: -98 --- Result: FAILED
Episode: 645 --- Time: 0.405 --- Reward: -98 --- Result: FAILED
Episode: 646 --- Time: 0.394 --- Reward: -99 --- Result: FAILED
Episode: 647 --- Time: 0.365 --- Reward: -99 --- Result: FAILED
Episode: 648 --- Time: 0.469 --- Reward: -99 --- Result: FAILED
Episode: 649 --- Time: 0.468 --- Reward: -95 --- Result: FAILED
Episode: 650 --- Time: 0.417 --- Reward: -100 --- Result: FAILED
Episode: 651 --- Time: 0.427 --- Reward: -100 --- Result: FAILED
Episode: 652 --- Time: 0.449 --- R

Episode: 765 --- Time: 0.429 --- Reward: -98 --- Result: FAILED
Episode: 766 --- Time: 0.396 --- Reward: -97 --- Result: FAILED
Episode: 767 --- Time: 0.373 --- Reward: -99 --- Result: FAILED
Episode: 768 --- Time: 0.412 --- Reward: -98 --- Result: FAILED
Episode: 769 --- Time: 0.425 --- Reward: -97 --- Result: FAILED
Episode: 770 --- Time: 0.626 --- Reward: -95 --- Result: FAILED
Episode: 771 --- Time: 0.403 --- Reward: -99 --- Result: FAILED
Episode: 772 --- Time: 0.433 --- Reward: -96 --- Result: FAILED
Episode: 773 --- Time: 0.339 --- Reward: -100 --- Result: FAILED
Episode: 774 --- Time: 0.370 --- Reward: -98 --- Result: FAILED
Episode: 775 --- Time: 0.475 --- Reward: -95 --- Result: FAILED
Episode: 776 --- Time: 0.403 --- Reward: -97 --- Result: FAILED
Episode: 777 --- Time: 0.416 --- Reward: -97 --- Result: FAILED
Episode: 778 --- Time: 0.396 --- Reward: -97 --- Result: FAILED
Episode: 779 --- Time: 0.403 --- Reward: -98 --- Result: FAILED
Episode: 780 --- Time: 0.385 --- Reward

Episode: 893 --- Time: 2.110 --- Reward: -44 --- Result: FAILED
Episode: 894 --- Time: 2.134 --- Reward: -60 --- Result: FAILED
Episode: 895 --- Time: 0.860 --- Reward: -82 --- Result: FAILED
Episode: 896 --- Time: 0.797 --- Reward: -81 --- Result: FAILED
Episode: 897 --- Time: 0.359 --- Reward: -100 --- Result: FAILED
Episode: 898 --- Time: 0.910 --- Reward: -93 --- Result: FAILED
Episode: 899 --- Time: 1.095 --- Reward: -66 --- Result: FAILED
Episode: 900 --- Time: 1.281 --- Reward: -63 --- Result: FAILED
Episode: 901 --- Time: 0.559 --- Reward: -86 --- Result: FAILED
Episode: 902 --- Time: 1.179 --- Reward: -65 --- Result: FAILED
Episode: 903 --- Time: 0.360 --- Reward: -101 --- Result: FAILED
Episode: 904 --- Time: 1.298 --- Reward: -69 --- Result: FAILED
Episode: 905 --- Time: 0.463 --- Reward: -93 --- Result: FAILED
Episode: 906 --- Time: 1.470 --- Reward: -63 --- Result: FAILED
Episode: 907 --- Time: 2.792 --- Reward: -20 --- Result: FAILED
Episode: 908 --- Time: 0.469 --- Rewar

Episode: 1021 --- Time: 1.151 --- Reward: -76 --- Result: FAILED
Episode: 1022 --- Time: 1.543 --- Reward: -44 --- Result: FAILED
Episode: 1023 --- Time: 1.678 --- Reward: -88 --- Result: FAILED
Episode: 1024 --- Time: 1.596 --- Reward: -48 --- Result: FAILED
Episode: 1025 --- Time: 4.779 --- Reward: 80 --- Result: FAILED
Episode: 1026 --- Time: 2.095 --- Reward: -42 --- Result: FAILED
Episode: 1027 --- Time: 2.175 --- Reward: -32 --- Result: FAILED
Episode: 1028 --- Time: 2.555 --- Reward: -4 --- Result: FAILED
Episode: 1029 --- Time: 3.027 --- Reward: -19 --- Result: FAILED
Episode: 1030 --- Time: 6.263 --- Reward: 101 --- Result: FAILED
Episode: 1031 --- Time: 5.404 --- Reward: 78 --- Result: FAILED
Episode: 1032 --- Time: 9.430 --- Reward: 202 --- Result: ----
Episode: 1033 --- Time: 13.063 --- Reward: 1 --- Result: FAILED
Episode: 1034 --- Time: 3.807 --- Reward: -31 --- Result: FAILED
Episode: 1035 --- Time: 4.625 --- Reward: 15 --- Result: FAILED
Episode: 1036 --- Time: 1.223 --

### Запустим еще на 20 эпох, чтобы убедиться, что все хорошо

In [14]:
for i_episode in range(20):

    start = time()
    state = env.reset()
    step = 0
    actual_reward = 0

    while True:
        if render:
            env.render()
        action = actor.choose_action(state)
        action = np.clip(np.random.normal(action, sigma), -1, 1)
        state_, reward, done, _ = env.step(action)
        
        actual_reward += reward
        step += 1
        
        result = "----"
        if reward==-100:
            result = "FAILED"
        elif env.unwrapped.hull.position[0] > (TERRAIN_LENGTH-TERRAIN_GRASS)*TERRAIN_STEP:
            result = "SUCCEEDED"
        if reward == -100: 
            reward = -2

        experience = np.hstack((state, action, [reward], state_))
        max_p = np.max(M.tree[-M.capacity:])
        M.remember(max_p, experience)

        if global_step.eval(sess) > memory_capacity/20:
            sigma = max([sigma*0.9999, sigma_min])
            idxs, batch_memory, weights = M.sample(batch_size)
            batch_state = batch_memory[:, :state_dim]
            batch_action = batch_memory[:, state_dim: state_dim + action_dim]
            batch_reward = batch_memory[:, -state_dim - 1: -state_dim]
            batch_state_ = batch_memory[:, -state_dim:]

            abs_td = critic.learn(batch_state, batch_action, batch_reward, batch_state_, weights)
            actor.learn(batch_state)
            
            for i in range(len(idxs)):
                idx = idxs[i]
                M.update(idx, abs_td[i])

        if done:
            if actual_reward > render_threshold:
                render = True
            else: 
                render = False
            
           
            print('Episode:', i_episode,
                  '--- Time: %.3f' % (time() - start),
                  '--- Reward: %i' % int(actual_reward),
                  '--- Result: %s' % result)
            break

        state = state_
        sess.run(increase_gs)

Episode: 0 --- Time: 18.542 --- Reward: 281 --- Result: SUCCEEDED
Episode: 1 --- Time: 18.981 --- Reward: 279 --- Result: SUCCEEDED
Episode: 2 --- Time: 19.697 --- Reward: 275 --- Result: SUCCEEDED
Episode: 3 --- Time: 18.073 --- Reward: 281 --- Result: SUCCEEDED
Episode: 4 --- Time: 19.733 --- Reward: 276 --- Result: SUCCEEDED
Episode: 5 --- Time: 7.444 --- Reward: 3 --- Result: FAILED
Episode: 6 --- Time: 9.394 --- Reward: -104 --- Result: ----
Episode: 7 --- Time: 9.674 --- Reward: -104 --- Result: ----
Episode: 8 --- Time: 9.597 --- Reward: -58 --- Result: ----
Episode: 9 --- Time: 6.176 --- Reward: 280 --- Result: SUCCEEDED
Episode: 10 --- Time: 19.171 --- Reward: 280 --- Result: SUCCEEDED
Episode: 11 --- Time: 20.454 --- Reward: 276 --- Result: SUCCEEDED
Episode: 12 --- Time: 18.861 --- Reward: 282 --- Result: SUCCEEDED
Episode: 13 --- Time: 19.228 --- Reward: 280 --- Result: SUCCEEDED
Episode: 14 --- Time: 19.430 --- Reward: 281 --- Result: SUCCEEDED
Episode: 15 --- Time: 19.190